# utils.py代码文件笔记

In [1]:
import os
import numpy as np
from math import sqrt
from scipy import stats
from torch_geometric.data import InMemoryDataset, DataLoader
from torch_geometric import data as DATA
import torch

class TestbedDataset(InMemoryDataset):                                                    #定一个InMemoryDataset的子类TestbedDataset，用于处理基于图形的机器学习测试数据集。
    def __init__(self, root='/tmp', dataset='davis',                                      
                 xd=None, xt=None, y=None, transform=None,
                 pre_transform=None, smile_graph=None):
        '''
        初始化接受以下参数：
        root: 数据集的根目录，如果未指定，则默认为 '/tmp'。
        dataset: 数据集名称，默认为 'davis'。
        xd: 一个 SMILES（分子smiles字符串）列表。
        xt: 编码后的目标（分类或one-hot编码）列表。
        y: 标签（亲和力）列表。
        transform: 可选的数据转换函数。
        pre_transform: 可选的预处理转换函数。
        smile_graph: 包含 SMILES 分子表示形式的图的字典。
        '''

        # root is required for save preprocessed data, default is '/tmp'                   #根目录需要保存预处理数据，默认路径为'/tmp'
        super(TestbedDataset, self).__init__(root, transform, pre_transform)               #集成父类属性和方法，并构造函数
        '''
        这段代码是调用父类InMemoryDataset的构造函数来初始化数据集。在Python中，子类可以通过调用父类的构造函数来继承父类的属性和方法。使用super函数可以方便地调用父类的构造函数，而不需要直接使用父类的名称。
        在这里，super(TestbedDataset, self)表示调用TestbedDataset的父类，即InMemoryDataset。__init__是InMemoryDataset中的构造函数
        通过传递root、transform和pre_transform参数来初始化InMemoryDataset实例
        因此，super(TestbedDataset, self).__init__(root, transform, pre_transform)的作用是调用父类的构造函数
        并使用给定的参数初始化TestbedDataset实例。这样，TestbedDataset可以继承InMemoryDataset的属性和方法，同时可以在其自身中定义其他的属性和方法
        '''


        # benchmark dataset, default = 'davis'
        self.dataset = dataset                                                              #定义属性变量dataset，这里是数据库名称
        if os.path.isfile(self.processed_paths[0]):                                         #如果有了数据预处理文件.pt则打印出来预处理数据发现，正在加载
            print(
                'Pre-processed data found: {}, loading ...'.format(self.processed_paths[0]))
            self.data, self.slices = torch.load(self.processed_paths[0])                    #将文件中数据加载属性变量data、slices中
        else:
            print(
                'Pre-processed data {} not found, doing pre-processing...'.format(self.processed_paths[0]))#如果该路径下没有数据预处理文件则进行数据处理
            '''
            打印出来是这样的效果：Pre-processed data found:data\processed\davis_train.pt,loading...
            '''
            self.process(xd, xt, y, smile_graph)                                            #利用process函数进行数据处理，process函数下边有定义
            self.data, self.slices = torch.load(self.processed_paths[0])                    #同上将数据加载到属性变量data、slices中       
'''
这是一个PyTorch中的自定义数据集类TestbedDataset，继承了InMemoryDataset类。
它允许用户使用指定的输入数据来进行初始化，包括化合物特征、蛋白质特征、标签以及SMILES图等，以及可选的变换操作和预处理操作。

TestbedDataset类的初始化方法__init__接受以下参数：
root：指定数据集的根目录，用于保存和加载预处理的数据，默认为/tmp。
dataset：指定使用的基准数据集，默认为davis。
xd：一个包含化合物特征的numpy数组。
xt：一个包含蛋白质特征的numpy数组。
y：一个包含标签的numpy数组。
transform：可选的数据变换操作，例如归一化、缩放等。
pre_transform：可选的预处理操作，例如数据清洗、降维等。
smile_graph：一个包含SMILES图的列表。

在初始化时，如果已经存在预处理的数据文件，则直接加载该文件，否则进行预处理并保存预处理结果。
TestbedDataset类实现了process方法，用于将输入的原始数据进行预处理并存储为PyTorch数据格式。
预处理后的数据包含Data对象和slices列表。Data对象包含节点特征、边信息和标签等信息。
slices列表包含了每个图的节点和边在Data对象中的起始和结束位置，用于切片访问数据。

'''

    #以下定义了这个类TestbedDataset中的几个方法
    @property
    def raw_file_names(self):                       #
        pass
        # return ['some_file_1', 'some_file_2', ...]

    @property
    def processed_file_names(self):
        return [self.dataset + '.pt']

    def download(self):
        # Download to `self.raw_dir`.
        pass

    def _download(self):
        pass

    def _process(self):
        if not os.path.exists(self.processed_dir):
            os.makedirs(self.processed_dir)
    '''
    这是一个PyTorch中的自定义数据集类TestbedDataset，继承了InMemoryDataset类。这里是类中的几个方法：

    raw_file_names：一个只读属性，用于指定原始数据文件的名称列表。
    processed_file_names：一个只读属性，用于指定预处理后数据文件的名称列表。
    download：将原始数据下载到指定的raw_dir目录中。
    _download：下载原始数据的实际方法，由download方法调用。
    _process：将原始数据处理为PyTorch可用的数据格式并保存到指定的processed_dir目录中。
    这些方法的具体实现并没有给出，因为它们是具体数据集需要自定义的部分。
    通常，如果原始数据文件已经存在，就不需要实现download和_download方法，只需要在raw_file_names属性中指定文件名称即可。
    而_process方法需要根据具体数据集的特点进行实现，将原始数据处理成Data对象，并将其保存为PyTorch可用的数据格式。
    在数据处理完成后，可以通过processed_file_names属性指定预处理后的数据文件名称。
    这里有些难理解，可以先放放，先把这个当成一个工具，感觉对于python小白有点难顶！你只要知道这块定义了一些方法是用来记录和存储数据的就行。
    '''




    #下来是关键需要好好理解一下
    # Customize the process method to fit the task of drug-target affinity prediction 设计专门适合于药物靶标结合亲和力预测任务的process方法
    # Inputs:
    # XD - list of SMILES, XT: list of encoded target (categorical or one-hot),
    # Y: list of labels (i.e. affinity)
    # Return: PyTorch-Geometric format processed data
    def process(self, xd, xt, y, smile_graph):                               #定义一个process方法，设置了几个参数xd，xt，y，smile_graph
        assert (len(xd) == len(xt) and len(xt) == len(y)), "The three lists must be the same length!"  #这是一个判断语句，这里意思是xd，xt，y的长度必须是相等的，否则返回语句"The three lists must be the same length!"
        data_list = []                                                       
        data_len = len(xd)
        for i in range(data_len):                                            #在每一个药物和靶标预测数据中进行循环遍历索引
            print('Converting SMILES to graph: {}/{}'.format(i+1, data_len)) #打印Converting SMILES to graph:百分比，意思是smiles正在转化为图数据的百分比
            smiles = xd[i]                                                   #对应每个循环索引定义变量smiles，target，labels，这三个对应一组数据，对这组数据进行预处理
            target = xt[i]
            labels = y[i]
            # convert SMILES to molecular representation using rdkit         #使用rdkit包将smiles字符串转化为分子表征
            c_size, features, edge_index = smile_graph[smiles]               #这是利用了smile_graph函数对每个smiles字符串的原子总数，特征矩阵(5个特征归一化的矩阵)，边的索引
            # make the graph ready for PyTorch Geometrics GCN algorithms:    #将数据转化为图卷积可以识别的数据
            GCNData = DATA.Data(x=torch.Tensor(features),                     
                                edge_index=torch.LongTensor(
                                    edge_index).transpose(1, 0),
                                y=torch.FloatTensor([labels]))
            '''
            这段代码利用Data这个类对所需数据进行转化为图卷积可以识别的数据
            具体有三个参数：
            x：表示节点的特征矩阵，其类型为 torch.Tensor，形状为 (num_nodes, num_features)，其中 num_nodes 是原子的总数量，num_features 是每个节点的特征向量的维度(5个特征放在一起44,11,11,1)这里都是78
            edge_index：表示节点之间的连接关系，其类型为 torch.LongTensor，形状为 (2, num_edges)，其中 num_edges 是边的数量。每列包含两个数字，表示一条边的两个端点在节点特征矩阵 x 中的索引。
            y：每组药物靶标结合亲和力值转化为FloatTensor格式
            '''
            GCNData.target = torch.LongTensor([target])
            '''
            这行代码主要是将一个名为"target"的变量转换成PyTorch的LongTensor格式，并且存储在名为"GCNData.target"的变量中。
            具体来说，它首先将"target"变量包装在一个Python列表中，然后将这个列表传递给torch.LongTensor()函数来创建一个PyTorch张量。
            在这个例子中，张量的形状是[1]，因为我们只传递了一个目标值。最后，这个张量被赋值给名为"GCNData.target"的变量，这个变量可以在后续的代码中使用。           
            '''
            GCNData.__setitem__('c_size', torch.LongTensor([c_size]))        #将c_size转化为longtensor数据
            # append graph, label and target sequence to data list
            data_list.append(GCNData)                                        #将x，edge_index，y，target，c_size添加到data.list列表中
    '''
    这是一个数据处理函数 process()，用于将给定的分子 SMILES 序列（xd）、目标序列（xt）和标签序列（y）转换为 PyTorch Geometric 中的 GCNData 对象列表，并保存到文件中。
    这个函数需要传递一个名为 smile_graph 的对象，这个对象包含所有 SMILES 序列对应的分子表示（拓扑结构），在这里被用于将每个 SMILES 转换为一个 PyTorch Geometric 的 GCNData 对象。

    该函数首先对输入数据进行一些检查，确保 xd、xt 和 y 的长度相同。然后它创建一个空的列表 data_list，用于存储 GCNData 对象。
    接下来，它使用 smile_graph 中的分子表示将每个 SMILES 转换为一个 GCNData 对象，并将其添加到 data_list 中。在这个过程中，它还将目标序列和标签序列分别添加到 GCNData 对象中。

    接下来，如果提供了 pre_filter 或 pre_transform 函数，则它将在 data_list 上应用这些函数。最后，它使用 self.collate() 方法将 data_list 转换为一个 Data 对象，并将其保存到磁盘上。

    需要注意的是，这个函数中的 GCNData 变量名被多次使用，其中一个是在 for 循环中定义的局部变量，而另一个是 torch_geometric.data.Data() 类的别名。
    这种命名约定可能会导致一些混淆，因此建议在编写代码时避免使用这样的变量名。
    '''


        if self.pre_filter is not None:
            data_list = [data for data in data_list if self.pre_filter(data)]
        '''
        这行代码是一个条件语句，用于在 data_list 列表中筛选出符合条件的数据对象。

        在这里，self.pre_filter 是一个可选的函数参数，如果它被传递了一个函数对象，那么这个函数将被用于对数据进行过滤。
        这个函数应该接受一个 GCNData 对象作为输入，返回一个布尔值，指示该对象是否应该保留在数据列表中。
        如果 self.pre_filter 没有被传递，那么它将为 None，这时不会执行任何过滤操作。

        条件语句中的代码 [data for data in data_list if self.pre_filter(data)] 使用了列表推导式，
        遍历 data_list 列表中的每个 GCNData 对象，并将其传递给 self.pre_filter() 函数进行过滤。
        只有当 self.pre_filter() 函数返回 True 时，当前对象才会被保留在新的列表中。最终，过滤后的 GCNData 对象列表被赋值给 data_list 变量，取代了原来的列表。
        '''

        if self.pre_transform is not None:
            data_list = [self.pre_transform(data) for data in data_list]       #同上但是这里我们都是默认设置的None没有对数据进行转化或者过滤
        print('Graph construction done. Saving to file.')
        data, slices = self.collate(data_list)
        # save preprocessed data:
        torch.save((data, slices), self.processed_paths[0])
        '''
        这段代码用于将处理后的数据保存到文件中，以便在以后的训练或测试过程中使用。

        print('Graph construction done. Saving to file.') 是一个简单的提示信息，告诉我们处理过程已经完成。

        data, slices = self.collate(data_list) 这行代码使用 torch_geometric.data.Batch.collate 方法将列表中的 GCNData 对象组合成一个 Batch 对象，使它们可以一起被输入到神经网络中进行训练或测试。
        Batch 对象是 GCNData 对象的批处理版本，它将多个 GCNData 对象合并成一个大型图形数据集，这样我们就可以将整个数据集一次性地输入到模型中进行训练或测试，从而加速模型的训练和测试过程。

        torch.save((data, slices), self.processed_paths[0]) 这行代码使用 torch.save 方法将处理后的数据保存到文件中，以便在以后的训练或测试过程中使用。
        在这里，我们使用了 self.processed_paths[0]，它是 Data 类中的一个属性，指示预处理后的数据应该被保存在哪个文件中。通常情况下，我们会将文件扩展名设置为 .pt，表示这是一个 PyTorch 格式的文件。
        '''


#下边是一些评价指标不做解释
def rmse(y, f):
    rmse = sqrt(((y - f)**2).mean(axis=0))
    return rmse


def mse(y, f):
    mse = ((y - f)**2).mean(axis=0)
    return mse


def pearson(y, f):
    rp = np.corrcoef(y, f)[0, 1]
    return rp


def spearman(y, f):
    rs = stats.spearmanr(y, f)[0]
    return rs


def ci(y, f):
    ind = np.argsort(y)
    y = y[ind]
    f = f[ind]
    i = len(y)-1
    j = i-1
    z = 0.0
    S = 0.0
    while i > 0:
        while j >= 0:
            if y[i] > y[j]:
                z = z+1
                u = f[i] - f[j]
                if u > 0:
                    S = S + 1
                elif u == 0:
                    S = S + 0.5
            j = j - 1
        i = i - 1
        j = i-1
    ci = S/z
    return ci   

IndentationError: unexpected indent (<ipython-input-1-dce85abd04b9>, line 72)

In [18]:
import torch
a=torch.load('D:\GraphDTA-master\data\processed\davis_train.pt')
a

(Data(x=[803206, 78], edge_index=[2, 1776072], y=[25046], target=[25046, 1000], c_size=[25046]),
 defaultdict(dict,
             {'x': tensor([     0,     25,     54,  ..., 803132, 803170, 803206]),
              'edge_index': tensor([      0,      54,     118,  ..., 1775914, 1775996, 1776072]),
              'y': tensor([    0,     1,     2,  ..., 25044, 25045, 25046]),
              'target': tensor([    0,     1,     2,  ..., 25044, 25045, 25046]),
              'c_size': tensor([    0,     1,     2,  ..., 25044, 25045, 25046])}))

In [ ]:
GCNData.target = torch.LongTensor([target])